In [ ]:
# get_info.py

from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.library.http import NBAStatsHTTP
import pandas as pd
import time

# Increase request timeout from 30s to 60s to avoid frequent timeouts
NBAStatsHTTP.TIMEOUT = 60

# Maximum number of players to fetch (None = all players)
MAX_PLAYERS = None  # Change to 15 for quick tests

# Maximum number of retries per player request
MAX_RETRIES = 3

# Convert height from string like "6-8" to inches
def feet_inches_to_inches(feet_str):
    try:
        if '-' in feet_str:
            feet, inches = feet_str.split('-')
            return int(feet) * 12 + int(inches)
        elif feet_str.isdigit():  # Handles cases like "7"
            return int(feet_str) * 12
    except:
        return None

# Retry wrapper for API requests
def get_player_info_with_retry(player_id):
    for attempt in range(MAX_RETRIES):
        try:
            return commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed (ID: {player_id}): {e}")
            time.sleep(2)
    print(f"❌ Giving up on player ID: {player_id}")
    return None

# Start fetching all active players
all_players = players.get_active_players()
info_data = []

for i, player in enumerate(all_players[:MAX_PLAYERS] if MAX_PLAYERS else all_players):
    player_id = player['id']
    name = player['full_name']
    print(f"📌 Processing player {i+1}: {name} (ID: {player_id})")

    info_obj = get_player_info_with_retry(player_id=player_id)
    if info_obj is None:
        continue

    try:
        df = info_obj.get_data_frames()[0]
        raw_height = df.loc[0, 'HEIGHT']
        weight = df.loc[0, 'WEIGHT']
        position = df.loc[0, 'POSITION']

        height_in = feet_inches_to_inches(raw_height)

        info_data.append({
            'player_id': player_id,
            'name': name,
            'height_in': height_in,
            'weight': weight,
            'position': position
        })

    except Exception as e:
        print(f"❌ Data parsing error, skipping {name}: {e}")
        continue

    time.sleep(0.5)  # Prevent getting rate-limited

# Save as CSV to Desktop
output_path = "/Users/james/Desktop/player_info.csv"
df_info = pd.DataFrame(info_data)
df_info.to_csv(output_path, index=False, quoting=1)
print(f"Player info saved successfully to: {output_path}")


📌 Processing player 1: Precious Achiuwa (ID: 1630173)
⚠️ Attempt 1 failed (ID: 1630173): ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
📌 Processing player 2: Steven Adams (ID: 203500)
📌 Processing player 3: Bam Adebayo (ID: 1628389)
📌 Processing player 4: Ochai Agbaji (ID: 1630534)
📌 Processing player 5: Santi Aldama (ID: 1630583)
📌 Processing player 6: Trey Alexander (ID: 1641725)
📌 Processing player 7: Nickeil Alexander-Walker (ID: 1629638)
📌 Processing player 8: Grayson Allen (ID: 1628960)
📌 Processing player 9: Jarrett Allen (ID: 1628386)
📌 Processing player 10: Jose Alvarado (ID: 1630631)
📌 Processing player 11: Kyle Anderson (ID: 203937)
📌 Processing player 12: Giannis Antetokounmpo (ID: 203507)
📌 Processing player 13: Cole Anthony (ID: 1630175)
📌 Processing player 14: OG Anunoby (ID: 1628384)
📌 Processing player 15: Taran Armstrong (ID: 1642379)
📌 Processing player 16: Deni Avdija (ID: 1630166)
📌 Processing player 17: Deandre Ayton (ID: 1629028)
📌